In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import load_model

import cv2
import platform
from tqdm import tqdm
import h5py
import yaml

Using TensorFlow backend.


In [5]:
# load keras model
model = load_model('../models/vgg_cnn_weights_25_0.10.hdf5')
image_size = (150,150)

In [6]:
# get the file names for all test images
file_names = os.listdir('../data/test-jpg')
test_images = [name.split('.')[0] for name in file_names]

In [9]:
# load test data function
def load_test_images(test_images):
    x_test = []
    for f in tqdm(test_images, miniters=10):
        img = cv2.imread('../data/test-jpg/{}.jpg'.format(f))
        x_test.append(cv2.resize(img, image_size))
    x_test = np.array(x_test, np.float16)/255.
    return x_test   

# test load_test_images
x_test = load_test_images(test_images)
print('x_test shape:      {}'.format(x_test.shape))

100%|██████████| 61191/61191 [02:51<00:00, 356.83it/s]


x_test shape:      (61191, 150, 150, 3)


In [10]:
with open('../label_maps.yml', 'r') as lablels:
    label_data = yaml.load(lablels)
inv_label_map = label_data['inv_label_map']
label_map = label_data['label_map']

In [11]:
def create_submission_file(test_images, p_test, threshold):
    p_test_threshold = p_test > threshold
    p_tags = []
    for p_single in p_test_threshold:
        p_tags.append(" ".join([inv_label_map[i] for i, tag_pred in enumerate(p_single) if tag_pred == 1]))
    submission_df = pd.DataFrame({'image_name': test_images})
    submission_df['tags'] = p_tags
    return submission_df
    
# test... requires x_test from above
p_test = model.predict(x_test, batch_size=10)
submission_df = create_submission_file(test_images[:10], p_test, 0.2)

submission_df

ValueError: Length of values does not match length of index

In [12]:
chunk_size = 12800
test_image_chunks = [test_images[x:x+chunk_size] for x in range(0, len(test_images), chunk_size)]
sub_list = []
for test_image_chunk in test_image_chunks:
    x_test = load_test_images(test_image_chunk)
    p_test = model.predict(x_test, batch_size=128)
    submission_df = create_submission_file(test_image_chunk, p_test, 0.2)
    sub_list.append(submission_df)
submission_df_final = pd.concat(sub_list)
submission_df_final.head()

100%|██████████| 9991/9991 [00:16<00:00, 611.75it/s]


,image_name,tags
0,test_38951,water haze clear primary
1,test_24774,cultivation agriculture clear primary
2,test_3232,haze primary
3,test_237,partly_cloudy primary
4,file_2684,road cultivation agriculture water clear primary


In [13]:
submission_df_final.to_csv('../submissions/submission_aws_vgg16.csv', index=False)

In [14]:
len(submission_df_final)

61191

In [ ]:
# attempt 1 - 0.839 (ankora simple keras)
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
# attempt 2 - 0.863 (keras blog link above)
